In [1]:
import theano
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

import pandas as pd
df = pd.read_csv('complaints.csv')
df.head()

g++.exe: error: Gupta\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.12-64\lazylinker_ext\lazylinker_ext.pyd: No such file or directory
g++.exe: error: C:\Users\Dikshant: No such file or directory
g++.exe: error: Gupta\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.12-64\lazylinker_ext\mod.cpp: No such file or directory



00001	#include <Python.h>
00002	#include "theano_mod_helper.h"
00003	#include "structmember.h"
00004	#include <sys/time.h>
00005	
00006	#if PY_VERSION_HEX >= 0x03000000
00007	#include "numpy/npy_3kcompat.h"
00008	#define PyCObject_AsVoidPtr  NpyCapsule_AsVoidPtr
00009	#define PyCObject_GetDesc  NpyCapsule_GetDesc
00010	#define PyCObject_Check NpyCapsule_Check
00011	#endif
00012	
00013	#ifndef Py_TYPE
00014	#define Py_TYPE(obj) obj->ob_type
00015	#endif
00016	
00017	/**
00018	
00019	TODO: 
00020	- Check max supported depth of recursion
00021	- CLazyLinker should add context information to errors caught during evaluation. Say what node we were on, add the traceback attached to the node.
00022	- Clear containers of fully-useed intermediate results if allow_gc is 1
00023	- Add timers for profiling
00024	- Add support for profiling space used.
00025	
00026	
00027	  */
00028	static double pytime(const struct timeval * tv)
00029	{
00030	  struct timeval t;
00031	  if (!tv)
00032	    {
00033	 

Exception: Compilation failed (return status=1): g++.exe: error: Gupta\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.12-64\lazylinker_ext\lazylinker_ext.pyd: No such file or directory. g++.exe: error: C:\Users\Dikshant: No such file or directory. g++.exe: error: Gupta\AppData\Local\Theano\compiledir_Windows-10-10.0.14393-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.12-64\lazylinker_ext\mod.cpp: No such file or directory. 

In [ ]:
def complaint_to_words(comp):
    
    words = RegexpTokenizer('\w+').tokenize(comp)
    num = RegexpTokenizer('\d+').tokenize(comp)
    words = [w for w in words if w not in num]
    words = [w.lower() for w in words]
    
    return words

In [ ]:
all_words = list()
for comp in df['Consumer complaint narrative']:
    for w in complaint_to_words(comp):
        all_words.append(w)

In [ ]:
len(set(all_words)), len(all_words)

In [5]:
index_dict = dict()
count = 0
for word in set(all_words):
    index_dict[word] = count
    count += 1

In [6]:
data_list = list()
for comp in df['Consumer complaint narrative']:
    l = list()
    for w in complaint_to_words(comp):
        l.append(index_dict[w])
    data_list.append(l)

In [7]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['Product'])
df['Target'] = le.transform(df['Product'])
df.head()

,Consumer complaint narrative,Product,Target
0,Received Capital One charge card offer XXXX. A...,Credit card,2
1,I do n't know how they got my cell number. I t...,Debt collection,4
2,I 'm a longtime member of Charter One Bank/RBS...,Credit card,2
3,"After looking at my credit report, I saw a col...",Credit reporting,3
4,I received a call from a XXXX XXXX from XXXX @...,Debt collection,4


In [8]:
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(df['Target'].values)
y_binary.shape

(105504L, 12L)

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(numpy.array(data_list), y_binary, 
    test_size=0.4, random_state=0)

In [25]:
type(X_train), type(X_test)

(numpy.ndarray, numpy.ndarray)

In [26]:
len(X_train[0]), len(X_train[1]), len(X_train[2])

(750, 750, 750)

In [27]:
# truncate and pad input sequences
max_review_length = 750
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [28]:
len(X_train[0]), len(X_train[1]), len(X_train[2])

(750, 750, 750)

In [31]:
from keras.layers import Dropout
from keras.layers import GRU

In [34]:
# create the model
top_words = 62943
embedding_vecor_length = 100
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(30, dropout_W=0.2, dropout_U=0.2))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 750, 100)      6294300     embedding_input_3[0][0]          
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 750, 100)      0           embedding_3[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 30)            15720       dropout_4[0][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 30)            0           lstm_1[0][0]                     
___________________________________________________________________________________________

In [32]:
# create the model
top_words = 62943
embedding_vecor_length = 100
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(GRU(32, dropout_W=0.2, dropout_U=0.2))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 750, 100)      6294300     embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 750, 100)      0           embedding_2[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 32)            12768       dropout_2[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 32)            0           gru_1[0][0]                      
___________________________________________________________________________________________

In [33]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 75.95%
